# Form Ilastik Masks to Labels
<div class="custom-button-row">
    <a 
        class="custom-button custom-download-button" href="../../../notebooks/05_segmentation/machine_learning/from_ilastik_masks_to_labels.ipynb" download>
        <i class="fas fa-download"></i> Download this Notebook
    </a>
    <a
    class="custom-button custom-download-button" href="https://colab.research.google.com/github/HMS-IAC/bobiac/blob/gh-pages/colab_notebooks/05_segmentation/machine_learning/from_ilastik_masks_to_labels.ipynb" target="_blank">
        <img class="button-icon" src="../../../_static/logo/icon-google-colab.svg" alt="Open in Colab">
        Open in Colab
    </a>
</div>

In [ ]:
# /// script
# requires-python = ">=3.12"
# dependencies = [
#     "matplotlib",
#     "ndv[jupyter,vispy]",
#     "scikit-image",
#     "scipy",
#     "tifffile",
#     "imagecodecs",
# ]
# ///

## <p class="alert alert-success">Description</p>

This notebook demonstrates how to convert the **semantic segmentation** generated by **Ilastik** (Simple Segmentation) into **instance segmentation**.

<div align="center"> <img src="../../../_static/images/ilastik/sem_to_inst.png" alt="Ilastik Logo" width="600"></div>

<br>

We first explore the type of data that we generated with Ilastik in the previous section ([**Machine Learning: Ilastik**](./machine_learning_with_ilastik.md)) and we then use them to generate labels, which can be used for instance segmentation (as we did in [classic segmentastion methods](../classic/classic.md)).

***TODO: UPDATE LINK TO CLASSIC SEGMENTATION METHODS SECTION***

### <p class="alert alert-info">Import libraries</p>

In [40]:
import tifffile
import ndv
from scipy import ndimage
from skimage import measure, segmentation, morphology, color
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np

### <p class="alert alert-info">Explore the Data</p>

Let's first load one of the *_Simple Segmentation.tif* files and explore the data that we generated with Ilastik. We want to know the type of data that we have before we can convert it to labels (instance segmentation).

Choose one of the *_Simple Segmentation.tif* files path and store it in a variable called `seg_path`.

In [ ]:
# set the path to one of the *_Simple Segmentation.tif files
seg_path = "../../../_static/images/ilastik/ilastik_simple_seg_example.tif"

We can use the `tifffile` library to read the tif file.

In [ ]:
# Load a mask
seg = tifffile.imread(seg_path)

What is the data type?

In [ ]:
print(seg.dtype)
print(type(seg))

With can now use the `ndv` library to display the data and **explore the pixel values**.

What are the values? Is this a [0 1] binary mask?

In [ ]:
# show the mask image
ndv.imshow(seg)

In [ ]:
viewer = ndv.imshow(seg)

In [ ]:
viewer.widget().children[1].snapshot()

By exploring the results we can notice that all **the pixels within the nuclei regions have a value of 1**, while the **background has a value of 2**.

This is because Ilastik assigns integer to classes starting from 1, based on the number and the order of the classes defined during the training phase. For instance, in our case, we defined two classes: the first class corresponds to the nuclei, and the second class corresponds to the background. As a result, the nuclei are labeled with the value 1 and the background with the value 2.

Remember that in order to generate labels form the data, **we need binary masks** with pixel values of **0 for the background** and **1 for our object of interest**, the nuclei.

How can we convert them to binary?

### <p class="alert alert-info">Data to Binary Masks</p>

In [ ]:
# Convert to a binary mask
# The nuclei pixels have all value 1, the seg image is a numpy array.
# We can create a new bunary mask by keeping only the pixels with value 1.
binary_mask = seg == 1

If we explore the binary masks with ndv, we can now see that the background has a value of 0 and the nuclei have a value of 1.

In [ ]:
ndv.imshow(binary_mask)

In [ ]:
viewer = ndv.imshow(seg)

In [ ]:
viewer.widget().children[1].snapshot()

### <p class="alert alert-info">Binary Masks to Labels</p>

Now that we have the binary mask, we can convert it to labels (instance segmentation) as we did in the [classic segmentastion methods](../classic/classic.md) section.

***TODO: UPDATE LINK TO CLASSIC SEGMENTATION METHODS SECTION***

In [ ]:
# compute the distance transform
distance = ndimage.distance_transform_edt(binary_mask)
# apply a Gaussian filter to smooth the distance transform
blurred_distance = ndimage.gaussian_filter(distance, sigma=4)
# find local maxima in the blurred distance transform
local_maxima = morphology.local_maxima(blurred_distance)
# use the local maxima to create seeds for the watershed algorithm
seeds = measure.label(local_maxima)
# apply the watershed algorithm to segment the image and get labels
labels = segmentation.watershed(-blurred_distance, seeds, mask=binary_mask)

We can now plot the labels and check the results using scikit-image's `label2rgb` function.

In [ ]:
img = color.label2rgb(labels, bg_label=0)
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(img, interpolation="nearest")
ax.axis("off")
plt.tight_layout()
plt.show()

We can also save the labels as a tif file using the `tifffile` library.

<p class="alert alert-info">
    <strong>NOTE:</strong> The <i>dtype</i> of the labels image is important because will determine the maximum number of labels that can be stored in the image. Infact, in a label image, each object is assigned a unique integer label, and the <i>dtype</i> determines the range of integers that can be used for labeling.
    <br>
    For instance, if we use <i>uint8</i>, the maximum number of labels is <i>255</i>, if we use <i>uint16</i>, the maximum number of labels is <i>65535</i> and so on.
</p>

In [38]:
tifffile.imwrite("watershed_segmentation.tif", labels.astype("uint16"))

### <p class="alert alert-info">Batch processing</p>

Since we now understand how to deal with the `Simple Segmentation` data from **Ilastik**, in order to obtain instance segmentation from all the *_Simple Segmentation.tif* files we generated, we can modify the batch script we used in the [classic segmentation methods](../classic/classic.md) section by simply adding the line of code where we select only the pixels with a value of 1 to only consider the nuclei.

***TODO: UPDATE LINK TO CLASSIC SEGMENTATION METHODS SECTION***

#### <p class="alert alert-warning"> <strong>✍️ Exercise: Batch Masks to Labels</strong></p>

Write a script that will convert all the *_Simple Segmentation.tif* images generated by Ilastik into labels images (instance segmentation) and save them as tif files.

In [ ]:
# make a function to create labels from a _Simple Segmentation.tif file
def ilastik_seg_to_labels(seg_image: np.ndarray, object_index: int) -> np.ndarray:
    """Convert an ilastik simpole segmentation image to labeled image.

    Parameters
    ----------
    seg_image : np.ndarray
        The input segmentation image generated by ilastik.
    object_index : int
        The index of the object to segment (e.g. 1 if your objects have value 1).
    """
    # convert to a binary mask
    binary_mask = seg_image == object_index
    # compute the distance transform
    distance = ndimage.distance_transform_edt(binary_mask)
    # apply a Gaussian filter to smooth the distance transform
    blurred_distance = ndimage.gaussian_filter(distance, sigma=4)
    # find local maxima in the blurred distance transform
    local_maxima = morphology.local_maxima(blurred_distance)
    # use the local maxima to create seeds for the watershed algorithm
    seeds = measure.label(local_maxima)
    # apply the watershed algorithm to segment the image and get labels
    return segmentation.watershed(-blurred_distance, seeds, mask=binary_mask)


# set the input directory to the path of the _Simple Segmentation.tif files
input_dir = Path("my_input_directory")
# set the output directory to save the labels
output_dir = Path("my_output_directory")

for seg_file in input_dir.glob("*_Simple Segmentation.tif"):
    seg_image = tifffile.imread(seg_file)
    labels = ilastik_seg_to_labels(seg_image, object_index=1)
    output_file = output_dir / f"{seg_file.stem}_labels.tif"
    tifffile.imwrite(output_file, labels.astype("uint16"))

print("DONE!")

DONE!
